# Chapter 4

We unify the pay-off and maturity inside a single class.

A First Solution:

In [7]:
from random import random
from math import exp, sqrt, log
import numpy as np
import pandas as pd

In [8]:
class VanillaOption():
    def __init__(self,ThePayOff_,Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self,Spot):
        return self.__ThePayOff(Spot)

In [9]:
def SimpleMonteCarlo3(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    Expiry = TheOption.GetExpiry()
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = np.random.normal()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean      

In [10]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

In [11]:
class PayOffCall(PayOff):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

In [12]:
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
callPrice = SimpleMonteCarlo3(TheOption,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.71, for 10000 paths


At this point we build a bridge to build an object between the pay-off and the option class to add extra functionalities like memory management and other features.

In [13]:
class PayOffBridge():
    def __init__(self,PayOff_):
        self.__PayOff = PayOff_
        
    def __del__(self):
        del self.__PayOff
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self.__PayOff = deepcopy(InnerPayOff)

    
    def __call__(self,Spot):
        return self.__PayOff(Spot)

In [14]:
class VanillaOption():
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)

In [15]:
callPayOff = PayOffBridge(PayOffCall(40))
TheOption = VanillaOption(callPayOff,0.5)
callPrice = SimpleMonteCarlo3(TheOption,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.67, for 10000 paths


In [17]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

In [18]:
class ParametersConstant(ParametersInner):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare

In [19]:
class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    

In [20]:
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare

In [21]:
class PayOffBridge():
        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
    


In [22]:
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

In [25]:
def SimpleMonteCarlo4(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    Expiry = TheOption.GetExpiry()
    
    variance = Vol.IntegralSquare(0,Expiry)
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r.Integral(0,Expiry) + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = np.random.normal()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = TheOption.OptionPayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r.Integral(0,Expiry))
    return mean    

In [26]:
callPayOff = PayOffCall(40)
TheOption = VanillaOption(callPayOff,0.5)
r = ParametersConstant(0.1)
Vol = ParametersConstant(0.2)
callPrice = SimpleMonteCarlo4(TheOption,42,Vol,r,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.69, for 10000 paths
